In [2]:
import numpy as np
import pandas as pd
import json

import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

from geopy.geocoders import Nominatim
!pip install geopy
!pip install html5lib
!pip install lxml
!pip install BeautifulSoup4

print('librariers are imported')


     |████████████████████████████████| 5.5MB 829kB/s eta 0:00:01     |█████████████████████████▉      | 4.5MB 5.2MB/s eta 0:00:01
     |████████████████████████████████| 122kB 6.5MB/s eta 0:00:01
librariers are imported


In [11]:
from pandas.io.html import read_html
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data = read_html(url, attrs={"class":"wikitable"})
print('Data downloaded!')

Data downloaded!


In [12]:
toronto_data[0]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [13]:
toronto_data[0].drop(toronto_data[0].index[toronto_data[0]['Borough'] == 'Not assigned'], inplace=True)

toronto_data[0] = toronto_data[0].reset_index(drop=True)

toronto_data[0]

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
toronto_data=toronto_data[0].groupby('Postal Code').agg(','.join)
#df_postcode = toronto_data.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
toronto_data.head()

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [16]:
df_postcode = toronto_data.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_postcode.replace({'Neighbourhood':'Not Assigned'},(df_postcode['Borough']))

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [17]:
df_postcode.shape

(103, 3)

In [18]:
url = 'http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/,'
url

'http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/,'

In [19]:
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
toronto_data = df_postcode.join(locgeo_df, on='Postal Code')
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
import geocoder # import geocoder
postal_code = toronto_data['Postal Code']
# Using while Statement
latitude = []longitude = []n = 0
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng
print('The geograpical coordinate of {} are {}, {}.'.format(postal_code[n], lat_
lng_coords[0], lat_lng_coords[1]))
    latitude.append(lat_lng_coords[0])    
    longitude.append(lat_lng_coords[1])    
    n = n + 1
    
toronto_data['Latitude'] = latitude
toronto_data['Longitude'] = longitude
toronto_data.head(12)

SyntaxError: invalid character in identifier (<ipython-input-20-3584e07f7f45>, line 1)

In [21]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [26]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto


NameError: name 'neighborhoods' is not defined